In [8]:
from neo4j import GraphDatabase, basic_auth
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "123456789"))

In [2]:
import pandas as pd
imdb = pd.read_csv('resources/imdb.csv')
imdb.head()

,Series_Title,Released_Year,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes
0,The Shawshank Redemption,1994,9.3,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110
1,The Godfather,1972,9.2,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367
2,The Dark Knight,2008,9.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232
3,The Godfather: Part II,1974,9.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952
4,12 Angry Men,1957,9.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845


In [9]:
cypher_query = '''
MERGE (m:Movie {id: $id, title: $title, year: $year, rating: $rating, votes: $votes})
MERGE (d:Director {name: $director})
MERGE (d)-[:HAS_DIRECTED]->(m)
MERGE (a1:Actor {name: $star1})
MERGE (a2:Actor {name: $star2})
MERGE (a3:Actor {name: $star3})
MERGE (a4:Actor {name: $star4})
MERGE (a1)-[:HAS_ACTED_IN]->(m)
MERGE (a2)-[:HAS_ACTED_IN]->(m)
MERGE (a3)-[:HAS_ACTED_IN]->(m)
MERGE (a4)-[:HAS_ACTED_IN]->(m)
'''

In [10]:
def create_movie(tx, id, title, year, rating, votes, director, star1, star2, star3, star4):
    tx.run(cypher_query, id = id, title = title, year = year,
    rating =  rating, votes = votes, director = director,
    star1 = star1, star2 = star2, star3 = star3, star4 = star4)

In [11]:
with driver.session(database="neo4j") as session:
    for i, row in imdb.iterrows():
        id = i
        title = row['Series_Title']
        year = row['Released_Year']
        rating = row['IMDB_Rating']
        votes = row['No_of_Votes']
        director = row['Director']
        star1 = row['Star1']
        star2 = row['Star2']
        star3 = row['Star3']
        star4 = row['Star4']
        session.write_transaction(create_movie, id, title, year,
        rating, votes, director, star1, star2, star3, star4)

driver.close()

/tmp/ipykernel_128071/3107879703.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_movie, id, title, year,


10 movies with most votes

In [12]:
cypher_query = '''
MATCH (m:Movie)
RETURN m.title, m.votes, m.rating
ORDER BY m.votes DESC
LIMIT 10'''

def get_result(tx):
    result = tx.run(cypher_query).data()
    return result

with driver.session(database="neo4j") as session:
    res = session.read_transaction(get_result)
driver.close()

res = pd.DataFrame(res)
res

/tmp/ipykernel_128071/2369182999.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  res = session.read_transaction(get_result)


,m.title,m.votes,m.rating
0,The Shawshank Redemption,2343110,9.3
1,The Dark Knight,2303232,9.0
2,Inception,2067042,8.8
3,Fight Club,1854740,8.8
4,Pulp Fiction,1826188,8.9
5,Forrest Gump,1809221,8.8
6,The Matrix,1676426,8.7
7,The Lord of the Rings: The Fellowship of the Ring,1661481,8.8
8,The Lord of the Rings: The Return of the King,1642758,8.9
9,The Godfather,1620367,9.2


3 best rated movies in whcih Elijah Wood has acted

In [15]:
cypher_query = '''
MATCH (m:Movie)<-[:HAS_ACTED_IN]-(a:Actor)
WHERE a.name = "Elijah Wood"
RETURN m.title, m.year, m.rating
ORDER BY m.rating DESC
LIMIT 3'''

with driver.session(database="neo4j") as session:
    res = session.read_transaction(get_result)
driver.close()
res = pd.DataFrame(res)
res

/tmp/ipykernel_128071/175357235.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  res = session.read_transaction(get_result)


,m.title,m.year,m.rating
0,The Lord of the Rings: The Return of the King,2003,8.9
1,The Lord of the Rings: The Fellowship of the Ring,2001,8.8
2,The Lord of the Rings: The Two Towers,2002,8.7


## Début du TD sur les graphes

Read files

In [22]:
edges = pd.read_csv('resources/stackoverflow_edges.csv', header=0)
nodes = pd.read_csv('resources/stackoverflow_nodes.csv', header=0)

In [23]:
edges.head()

,source,target,weight
0,azure,.net,20.933192
1,sql-server,.net,32.322524
2,asp.net,.net,48.407030
3,entity-framework,.net,24.370903
4,wpf,.net,32.350925


In [24]:
nodes.head()

,name
0,html
1,css
2,hibernate
3,spring
4,ruby


Create DataBase

In [25]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "pass4database"))

Nodes

In [27]:
cypher_query = '''
MERGE (n:Node {tag: $tag, name: $name})
'''

def create_nodes(tx, tag, name):
    tx.run(cypher_query, tag = tag, name = name)

with driver.session(database="neo4j") as session:
    for i, row in nodes.iterrows():
        tag = i
        name = row['name']
        session.write_transaction(create_nodes, tag, name)

driver.close()

/tmp/ipykernel_128071/157597023.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, tag, name)


Edges

https://neo4j.com/docs/cypher-manual/current/clauses/create/

In [29]:
cypher_query_edges = '''
MATCH
  (a:Node),
  (b:Node)
WHERE a.name = 'A' AND b.name = 'B'
CREATE (a)-[r:RELTYPE]->(b)
RETURN type(r)
'''

def create_edges(tx, source, target):
    tx.run(cypher_query_edges, source = source, target = target)

with driver.session(database="neo4j") as session:
    for i, row in edges.iterrows():
        tag = i
        source = row['source']
        target = row['target']
        session.write_transaction(create_nodes, source, target)

driver.close()

/tmp/ipykernel_128071/2981543972.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, source, target)
